# 引入套件

In [22]:
import pandas as pd

# Titanic 資料探索

## 讀取 csv 檔

In [23]:
train_df = pd.read_csv('titanic_data/train.csv')
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


<img src="image/data_description.PNG" width="100%">

## 觀察一下資料狀況

In [24]:
# 可以觀察缺失值狀況和資料型態
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [25]:
# 可以觀察數值型資料大約的分布狀況
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [26]:
# 將 Pclass 轉為類別型資料
train_df["Pclass"] = train_df["Pclass"].astype("object")

In [27]:
# 可以觀察類別型資料大約的分布狀況
train_df.describe(include=['O'])

,Pclass,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,891,204,889
unique,3,891,2,681,147,3
top,3,"Meek, Mrs. Thomas (Annie Louise Rowley)",male,1601,G6,S
freq,491,1,577,7,4,644


## 做樞紐分析驗證我們的猜測

In [28]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [29]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [30]:
# 補充

# 用 pivot_table 也可以達到跟上面一樣的效果
# train_df.pivot_table(index=['Pclass'], values='Survived', margins=True, aggfunc=['mean', 'count'])

# Titanic 特徵工程

## 補缺失值

In [31]:
# 算出 Embarked 最常出現的值
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [32]:
# Embarked 的缺失值用最常出現的值來補
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [33]:
# Fare 的缺失值用 Fare 的中位數來補
train_df['Fare'] = train_df['Fare'].fillna(train_df['Fare'].dropna().median())
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [34]:
# Age 的缺失值用 Sex 跟 Pclass 群聚後的 Age 平均值來補
train_df['Age'] = train_df['Age'].fillna(train_df.groupby(['Sex', 'Pclass'])['Age'].transform('mean'))
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,26.507589,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,NaN,C


## 丟棄缺失值過多或是對預測目標不會有幫助的 column

In [35]:
print(f"丟棄前: {train_df.shape}")
train_df = train_df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)
print(f"丟棄後: {train_df.shape}")

丟棄前: (891, 12)
丟棄後: (891, 8)


In [36]:
# 補充

# dropna 可以直接把有出現 na 值的 row 都丟掉，可是如果大部分的 row 都有缺失值就不建議這麼做
# train_df = train_df.dropna()

## 創造新的特徵

In [37]:
# FamilySize 代表這個乘客的家庭成員數量
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [38]:
# IsAlone 代表這個乘客是不是自己搭乘
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1
train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [39]:
# 如果覺得新做出來的特徵的資訊量足夠代表原本的特徵，也可以把原本的特徵丟掉
# train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
# train_df.head()

In [40]:
# Age*Class 代表這個乘客的 Age 跟 Pclass 相乘
train_df['Age*Class'] = train_df.Age * train_df.Pclass
train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

,Age*Class,Age,Pclass
0,66,22.000000,3
1,38,38.000000,1
2,78,26.000000,3
3,35,35.000000,1
4,105,35.000000,3
5,79.5228,26.507589,3
6,54,54.000000,1
7,6,2.000000,3
8,81,27.000000,3
9,28,14.000000,2


## 將原本是字串的類別型資料做編碼

In [21]:
train_df['Sex'] = train_df['Sex'].map({'female': 0, 'male': 1})
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Age*Class
0,0,3,1,22.000000,1,0,7.2500,S,2,0,66
1,1,1,0,38.000000,1,0,71.2833,C,2,0,38
2,1,3,0,26.000000,0,0,7.9250,S,1,1,78
3,1,1,0,35.000000,1,0,53.1000,S,2,0,35
4,0,3,1,35.000000,0,0,8.0500,S,1,1,105
5,0,3,1,26.507589,0,0,8.4583,Q,1,1,79.5228
6,0,1,1,54.000000,0,0,51.8625,S,1,1,54
7,0,3,1,2.000000,3,1,21.0750,S,5,0,6
8,1,3,0,27.000000,0,2,11.1333,S,3,0,81
9,1,2,0,14.000000,1,0,30.0708,C,2,0,28


In [22]:
train_df['Embarked'] = train_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Age*Class
0,0,3,1,22.000000,1,0,7.2500,0,2,0,66
1,1,1,0,38.000000,1,0,71.2833,1,2,0,38
2,1,3,0,26.000000,0,0,7.9250,0,1,1,78
3,1,1,0,35.000000,1,0,53.1000,0,2,0,35
4,0,3,1,35.000000,0,0,8.0500,0,1,1,105
5,0,3,1,26.507589,0,0,8.4583,2,1,1,79.5228
6,0,1,1,54.000000,0,0,51.8625,0,1,1,54
7,0,3,1,2.000000,3,1,21.0750,0,5,0,6
8,1,3,0,27.000000,0,2,11.1333,0,3,0,81
9,1,2,0,14.000000,1,0,30.0708,1,2,0,28


In [ ]:
# 補充

# 也可以改用 get_dummies 做 one hot encoding
# train_df = pd.get_dummies(train_df)
# train_df.head(10)

## 將連續型資料分配到各個區間，變成類別型資料

In [23]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.336714
2,"(32.252, 48.168]",0.412844
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [24]:
train_df.loc[ train_df['Age'] <= 16, 'Age'] = 0
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 32), 'Age'] = 1
train_df.loc[(train_df['Age'] > 32) & (train_df['Age'] <= 48), 'Age'] = 2
train_df.loc[(train_df['Age'] > 48) & (train_df['Age'] <= 64), 'Age'] = 3
train_df.loc[ train_df['Age'] > 64, 'Age'] = 4

In [25]:
train_df = train_df.drop(['AgeBand'], axis=1)
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Age*Class
0,0,3,1,1.0,1,0,7.2500,0,2,0,66
1,1,1,0,2.0,1,0,71.2833,1,2,0,38
2,1,3,0,1.0,0,0,7.9250,0,1,1,78
3,1,1,0,2.0,1,0,53.1000,0,2,0,35
4,0,3,1,2.0,0,0,8.0500,0,1,1,105
5,0,3,1,1.0,0,0,8.4583,2,1,1,79.5228
6,0,1,1,3.0,0,0,51.8625,0,1,1,54
7,0,3,1,0.0,3,1,21.0750,0,5,0,6
8,1,3,0,1.0,0,2,11.1333,0,3,0,81
9,1,2,0,0.0,1,0,30.0708,1,2,0,28


In [26]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [27]:
train_df.loc[ train_df['Fare'] <= 7.91, 'Fare'] = 0
train_df.loc[(train_df['Fare'] > 7.91) & (train_df['Fare'] <= 14.454), 'Fare'] = 1
train_df.loc[(train_df['Fare'] > 14.454) & (train_df['Fare'] <= 31), 'Fare']   = 2
train_df.loc[ train_df['Fare'] > 31, 'Fare'] = 3
train_df['Fare'] = train_df['Fare']

In [28]:
train_df = train_df.drop(['FareBand'], axis=1)
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Age*Class
0,0,3,1,1.0,1,0,0.0,0,2,0,66
1,1,1,0,2.0,1,0,3.0,1,2,0,38
2,1,3,0,1.0,0,0,1.0,0,1,1,78
3,1,1,0,2.0,1,0,3.0,0,2,0,35
4,0,3,1,2.0,0,0,1.0,0,1,1,105
5,0,3,1,1.0,0,0,1.0,2,1,1,79.5228
6,0,1,1,3.0,0,0,3.0,0,1,1,54
7,0,3,1,0.0,3,1,2.0,0,5,0,6
8,1,3,0,1.0,0,2,1.0,0,3,0,81
9,1,2,0,0.0,1,0,2.0,1,2,0,28


# 將整理好的資料保存下來

In [29]:
# 最後確認一下資料有沒有什麼問題
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    object 
 2   Sex         891 non-null    int64  
 3   Age         891 non-null    float64
 4   SibSp       891 non-null    int64  
 5   Parch       891 non-null    int64  
 6   Fare        891 non-null    float64
 7   Embarked    891 non-null    int64  
 8   FamilySize  891 non-null    int64  
 9   IsAlone     891 non-null    int64  
 10  Age*Class   891 non-null    object 
dtypes: float64(2), int64(7), object(2)
memory usage: 76.7+ KB


In [30]:
train_df.to_excel("titanic_data/train.xlsx", sheet_name = "sheet_1",index = False)

# 練習二
## 請建立一個新的 ipynb 檔，讀取 test.csv 檔案，試著根據前面有教到的方法，自己處理看看缺失值，最後使用 DataFrame 的 info 確定各欄位都沒有缺失值。